In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import train_test_split

# Exploración de información

In [ ]:
data = pd.read_csv('data_train(in).csv')

In [ ]:
data.head()

,user_id,tweet_id,tweet_text,class
0,user0001,0d3ed29586ce,Cheesecake saludable sin azúcar y sin lactosa ...,control
1,user0002,c3cf897a495b,ser como ellas ♡♡\n #HastaLosHuesos,anorexia
2,user0003,5041d85c45c6,"Comida Real o , la clave para estar más sana, ...",control
3,user0004,d18285d3c7ec,Entre el cambio de hora y la bajada de las #te...,control
4,user0005,4d81892f3217,Hace mucho tiempo no sentía mi cuerpo tan frío,anorexia


In [ ]:
data.tweet_text.value_counts()

,count
tweet_text,
1er día\n* Dos latas de coca cola light \n* Litro & medio de agua pura\n* Tres tazas de té sin azúcar...\n\n #ayuno #Ana #Princesa,1
Cheesecake saludable sin azúcar y sin lactosa con mermerlada casera de moras,1
ser como ellas ♡♡\n #HastaLosHuesos,1
"Comida Real o , la clave para estar más sana, delgada y feliz \n\nhttps://www.modalia.es/estilo-de-vida/15192-comida-real-o-real-food-la-clave-para-estar-mas-sana-delgada-y-feliz.html …",1
"Entre el cambio de hora y la bajada de las #temperaturas estamos destemplados, por eso os recomendamos un buen plato de #fabada #casera para entonar el cuerpo #decasa #platoscuchara #madridrio #gastro #arganzuela",1
...,...
Cambiar la comida por el alcohol,1
Norex Anfepromona un potente medicamento para dejar de comer en exceso\nADELGAZA SIN HAMBRE\n#TusClásicos #SabadoDeGanarSeguidores #EsPuenteYTodos #bellezasuperior\n#TwitterBestFandom\n#SoompiAwards\n#ConvoyVL19\n#20anosVL19 #Tampico #Dieta #Obesidad #Madero #Tamaulipas #Fit,1
"Los huesos que se me notan, son la copa, la medalla, por haberte ganado la batalla.\n.",1


# Limpieza de datos

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Simple cleaning
def clean_text(text):
    text = re.sub(r'(?<!^)(?=[A-ZÁÉÍÓÚÑÜ])', ' ', text)
    text = str(text).lower() # minúsculas
    text = re.sub(r"http\S+|www\S+|https\S+", '', text) # eliminar URLs
    text = re.sub(r"[^a-záéíóúñü\s]", '', text) # eliminar puntuación y números
    text = re.sub(r"\s+", ' ', text).strip() # eliminar espacios múltiples
    tokens = text.split()
    return ' '.join(tokens)

data['clean_text'] = data['tweet_text'].apply(clean_text)


In [ ]:
data.head()

,user_id,tweet_id,tweet_text,class,clean_text
0,user0001,0d3ed29586ce,Cheesecake saludable sin azúcar y sin lactosa ...,control,cheesecake saludable sin azúcar y sin lactosa ...
1,user0002,c3cf897a495b,ser como ellas ♡♡\n #HastaLosHuesos,anorexia,ser como ellas hasta los huesos
2,user0003,5041d85c45c6,"Comida Real o , la clave para estar más sana, ...",control,comida real o la clave para estar más sana del...
3,user0004,d18285d3c7ec,Entre el cambio de hora y la bajada de las #te...,control,entre el cambio de hora y la bajada de las tem...
4,user0005,4d81892f3217,Hace mucho tiempo no sentía mi cuerpo tan frío,anorexia,hace mucho tiempo no sentía mi cuerpo tan frío


In [ ]:
!python -m spacy download es_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 48.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

nlp = spacy.load('es_core_news_sm')

def clean_text_lemmatization(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"#(\w+)", r"\1", text)
    text = re.sub(r"[^a-záéíóúñü\s]", '', text)
    text = re.sub(r"\s+", ' ', text).strip()
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(lemmatized_tokens)

data['clean_text_lemma'] = data['tweet_text'].apply(clean_text_lemmatization)


In [ ]:
data.head()

,user_id,tweet_id,tweet_text,class,clean_text,clean_text_lemma
0,user0001,0d3ed29586ce,Cheesecake saludable sin azúcar y sin lactosa ...,control,cheesecake saludable sin azúcar y sin lactosa ...,cheesecakir saludable azúcar lactosa mermerlad...
1,user0002,c3cf897a495b,ser como ellas ♡♡\n #HastaLosHuesos,anorexia,ser como ellas hasta los huesos,hastaloshueso
2,user0003,5041d85c45c6,"Comida Real o , la clave para estar más sana, ...",control,comida real o la clave para estar más sana del...,comida real clave sano delgado feliz
3,user0004,d18285d3c7ec,Entre el cambio de hora y la bajada de las #te...,control,entre el cambio de hora y la bajada de las tem...,cambio hora bajada temperatura destemplado rec...
4,user0005,4d81892f3217,Hace mucho tiempo no sentía mi cuerpo tan frío,anorexia,hace mucho tiempo no sentía mi cuerpo tan frío,tiempo sentir cuerpo frío


In [ ]:
X_text = data['clean_text_lemma']


In [ ]:
X_text

,clean_text_lemma
0,cheesecakir saludable azúcar lactosa mermerlad...
1,hastaloshueso
2,comida real clave sano delgado feliz
3,cambio hora bajada temperatura destemplado rec...
4,tiempo sentir cuerpo frío
...,...
1495,desayuno sano ricocomidasán metodohappy
1496,perfección sábado án abandona thinspo thinspir...
1497,comer devolver caloría tengoundeseoyes perfeccion
1498,entender seguir soltero cocino masterchef demi...


In [ ]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
avg_letters = data['clean_text_lemma'].apply(lambda x: len(str(x).replace(" ", ""))).mean()
print(f"Promedio de letras por tweet: {avg_letters:.2f}")

Promedio de letras por tweet: 71.95


In [ ]:
data['class'] = data['class'].map({'control': 0, 'anorexia': 1})

In [ ]:
X_text = data['clean_text_lemma']
y = data['class']

In [ ]:
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(X_text)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
data.head()

,user_id,tweet_id,tweet_text,class,clean_text,clean_text_lemma
0,user0001,0d3ed29586ce,Cheesecake saludable sin azúcar y sin lactosa ...,0,cheesecake saludable sin azúcar y sin lactosa ...,cheesecakir saludable azúcar lactosa mermerlad...
1,user0002,c3cf897a495b,ser como ellas ♡♡\n #HastaLosHuesos,1,ser como ellas hasta los huesos,hastaloshueso
2,user0003,5041d85c45c6,"Comida Real o , la clave para estar más sana, ...",0,comida real o la clave para estar más sana del...,comida real clave sano delgado feliz
3,user0004,d18285d3c7ec,Entre el cambio de hora y la bajada de las #te...,0,entre el cambio de hora y la bajada de las tem...,cambio hora bajada temperatura destemplado rec...
4,user0005,4d81892f3217,Hace mucho tiempo no sentía mi cuerpo tan frío,1,hace mucho tiempo no sentía mi cuerpo tan frío,tiempo sentir cuerpo frío


In [ ]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=42 , eval_metric='logloss')
clf_xgb.fit(X_train, y_train)
preds_xgb = clf_xgb.predict(X_test)
accuracy_xgb = float(np.sum(preds_xgb == y_test)) / y_test.shape[0]
print('Accuracy de XGBoost:', round(accuracy_xgb, 4))

Accuracy de XGBoost: 0.7567


In [ ]:
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np

modelos = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(max_depth=None, max_features='log2', min_samples_split= 2, n_estimators= 200, random_state=42),
    "Support Vector Machine": SVC(C=1, class_weight=None, gamma='scale', kernel='rbf'),
    "XGBoost": xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, seed=42, eval_metric='logloss', learning_rate=0.1, max_depth=10, scale_pos_weight=1)
}

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd

resultados = []

for nombre, modelo in modelos.items():
    print(f"\nEvaluando {nombre}...")
    modelo.fit(X_train, y_train)
    predicciones = modelo.predict(X_test)

    acc = accuracy_score(y_test, predicciones)
    prec = precision_score(y_test, predicciones, zero_division=0)
    rec = recall_score(y_test, predicciones, zero_division=0)
    f1 = f1_score(y_test, predicciones, zero_division=0)
    cm = confusion_matrix(y_test, predicciones)

    resultados.append({
        'Modelo': nombre,
        'Accuracy': round(acc, 4),
        'Precision': round(prec, 4),
        'Recall': round(rec, 4),
        'F1-score': round(f1, 4)
    })

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")
    print("Confusion matrix:")
    print(cm)

# Mostrar resumen en tabla
df_resultados = pd.DataFrame(resultados)
display(df_resultados)




Evaluando Logistic Regression...
Accuracy: 0.8933
Precision: 0.8848
Recall: 0.9182
F1-score: 0.9012
Confusion matrix:
[[122  19]
 [ 13 146]]

Evaluando Decision Tree...
Accuracy: 0.8167
Precision: 0.8514
Recall: 0.7925
F1-score: 0.8208
Confusion matrix:
[[119  22]
 [ 33 126]]

Evaluando Random Forest...
Accuracy: 0.8700
Precision: 0.8333
Recall: 0.9434
F1-score: 0.8850
Confusion matrix:
[[111  30]
 [  9 150]]

Evaluando Support Vector Machine...
Accuracy: 0.9000
Precision: 0.8909
Recall: 0.9245
F1-score: 0.9074
Confusion matrix:
[[123  18]
 [ 12 147]]

Evaluando XGBoost...
Accuracy: 0.8333
Precision: 0.7824
Recall: 0.9497
F1-score: 0.8580
Confusion matrix:
[[ 99  42]
 [  8 151]]


,Modelo,Accuracy,Precision,Recall,F1-score
0,Logistic Regression,0.8933,0.8848,0.9182,0.9012
1,Decision Tree,0.8167,0.8514,0.7925,0.8208
2,Random Forest,0.8700,0.8333,0.9434,0.8850
3,Support Vector Machine,0.9000,0.8909,0.9245,0.9074
4,XGBoost,0.8333,0.7824,0.9497,0.8580


In [ ]:
# Fine tuning
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [ ]:
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2']
}

grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_rf,
    scoring='f1',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_rf.fit(X_train, y_train)
print("Mejores parámetros RF:", grid_rf.best_params_)
print("Mejor F1-score RF:", grid_rf.best_score_)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores parámetros RF: {'max_depth': None, 'max_features': 'log2', 'min_samples_split': 2, 'n_estimators': 200}
Mejor F1-score RF: 0.895622786588184


In [ ]:
param_grid_xgb = {
    'n_estimators': [50, 100],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.3],
    'scale_pos_weight': [1, 2, 5]  # útil si tu clase está desbalanceada
}

grid_xgb = GridSearchCV(
    estimator=xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False, seed=42),
    param_grid=param_grid_xgb,
    scoring='f1',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_xgb.fit(X_train, y_train)
print("Mejores parámetros XGBoost:", grid_xgb.best_params_)
print("Mejor F1-score XGBoost:", grid_xgb.best_score_)


Fitting 5 folds for each of 54 candidates, totalling 270 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:16:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parámetros XGBoost: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'scale_pos_weight': 1}
Mejor F1-score XGBoost: 0.8548677333480296


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'class_weight': [None, 'balanced']
}

grid_svm = GridSearchCV(
    estimator=SVC(),
    param_grid=param_grid_svm,
    scoring='f1',
    cv=5,
    verbose=1,
    n_jobs=-1
)

grid_svm.fit(X_train, y_train)

print("Mejores parámetros SVM:", grid_svm.best_params_)
print("Mejor F1-score SVM:", grid_svm.best_score_)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores parámetros SVM: {'C': 1, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}
Mejor F1-score SVM: 0.8840432286026608


Checar n grams, pasar los datos por sentiment analysis (pasar el resultado a columna y después evaular), probar con embeddings por token, chunk y texto porque tf idf sólo es un BoW mejorado